In [1]:
!pip install langchain langchain-classic langchain-community langchain-huggingface langchain-unstructured unstructured unstructured-client "unstructured[html]" chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip freeze > requirments.txt

In [6]:
!unzip chroma_db.zip -d /content/chroma_db

Archive:  chroma_db.zip
   creating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/
  inflating: /content/chroma_db/chroma.sqlite3  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/index_metadata.pickle  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/link_lists.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/header.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/length.bin  
  inflating: /content/chroma_db/0c523f6f-0d97-4701-aead-5a1853544eb2/data_level0.bin  


In [2]:
import requests
from bs4 import BeautifulSoup , Comment
import tempfile
import os
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import Chroma
import re

# get the html content from the research cluster page 'https://www.iitism.ac.in/research-cluster'

In [3]:
def load_html(url):
    if not url:
        return ""

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        response = requests.get(
            url,
            headers=headers,
            timeout=10,
            verify=False
        )
        response.raise_for_status()  # raises HTTPError for 4xx/5xx
        return response.text

    except requests.exceptions.RequestException as e:
        print(f"Error loading {url}: {e}")
        raise

In [4]:
def remove_comments(soup):
  for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
    comment.extract()

In [5]:
from google.colab import userdata

embedding_model = HuggingFaceEndpointEmbeddings(
    model = 'sentence-transformers/all-MiniLM-L6-v2',
    huggingfacehub_api_token=userdata.get('HF_TOKEN')
)

In [7]:
vector_store = Chroma(
  embedding_function=embedding_model,
  persist_directory="chroma_db",
  collection_name="collection_research_guide"
)

/tmp/ipython-input-1021428914.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store = Chroma(


In [9]:
def get_structured_docs(docs , url):

  # Prepare structured document array
  structured_docs = []

  current_title = None
  current_header = None

  # Categories to treat as titles
  TITLE_CATEGORIES = {"uncategorizedtext" , "title", "pagetitle", "main title" , "narrativetext"}

  # Categories to treat as headers
  HEADER_CATEGORIES = {"header", "header1", "header2", "header3", "header4", "header5", "header6", "subheading"}

  # Categories to ignore completely
  IGNORE_CATEGORIES = {"figure", "image" }

  for d in docs:
      category = d.metadata.get("category", "").lower()
      text = d.page_content.strip()

      if not text:
          continue

      if category in TITLE_CATEGORIES:
          current_title = text
          current_header = None  # reset header when new title
      elif category in HEADER_CATEGORIES:
          current_header = text
      elif category in IGNORE_CATEGORIES:
          continue  # skip figures/images
      else:
          # Other categories: paragraph, list, table, link, code, quote, etc.
          structured_docs.append(
              Document(
                  page_content=f" Title-{current_title or "Not available"} , heading-{current_header or "Not available"} , url-{url or "Not available"} \n\n description - {text}",
                  metadata={
                      "title": current_title,
                      "header": current_header,
                      "url": url or "Not available",
                  }
              )
          )

  return structured_docs

# prearing Links

In [10]:
html_content = load_html("https://www.iitism.ac.in/research-cluster")

soup = BeautifulSoup(html_content, "lxml")

for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
    comment.extract()

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [11]:
divs = soup.find("section" , class_ = "rts-about-university").find("div" , class_="container").find("div" , class_="link-holder").find_all("a")

links = []

for d in divs :
  links.append(d["href"])

In [12]:
links

['https://www.iitism.ac.in/research-cluster-applied-geology',
 'https://www.iitism.ac.in/research-cluster-applied-geophysics',
 'https://www.iitism.ac.in/research-cluster-chemical',
 'https://www.iitism.ac.in/research-cluster-chemistry-and-chemical-biology',
 'https://www.iitism.ac.in/research-cluster-civil',
 'https://www.iitism.ac.in/research-cluster-cse',
 'https://www.iitism.ac.in/research-cluster-ee',
 'https://www.iitism.ac.in/research-cluster-electronics-engineering',
 'https://www.iitism.ac.in/research-cluster-environmental-science-and-engineering',
 'https://www.iitism.ac.in/research-cluster-fuel-minerals-and-metallurgical-engineering',
 'https://www.iitism.ac.in/research-cluster-hss',
 'https://www.iitism.ac.in/research-cluster-msie',
 'https://www.iitism.ac.in/research-cluster-mnc',
 'https://www.iitism.ac.in/research-cluster-mechanical-engineering',
 'https://www.iitism.ac.in/research-cluster-mining-engineering',
 'https://www.iitism.ac.in/research-cluster-petroleum',
 'htt

# scrap each research cluster page


In [13]:
for link in links:

  html_content = load_html(link)

  if not html_content:
        continue

  soup = BeautifulSoup(html_content, "lxml")

  # remove comments from html
  remove_comments(soup)

  # filter only the body of the html content
  section = soup.find("section" , class_=["rts-about-university", "rts-section-padding"])

  if section is None:
        continue

  # store the html to a tmep file
  with tempfile.NamedTemporaryFile("w", suffix=".html", delete=False, encoding="utf-8") as tmp_file:
    tmp_file.write(str(section))
    tmp_path = tmp_file.name

  print("Temp file path:", tmp_path)

  # load html using Unstructure HTML loader
  loader = UnstructuredHTMLLoader(file_path=tmp_path, mode="elements")
  docs = loader.load()

  # remove the temp file
  os.remove(tmp_path)

  # get structured_docs from html using unstructed html loader
  structured_docs = get_structured_docs(docs , link)

  if not structured_docs:
        continue

  vector_store.add_documents(structured_docs)
  vector_store.persist()
  print(f"Added {len(structured_docs)} docs to db {link}")


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmpgeawu3at.html


/tmp/ipython-input-3733806387.py:40: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


Added 18 docs to db https://www.iitism.ac.in/research-cluster-applied-geology


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Temp file path: /tmp/tmp0ybp990i.html
Added 40 docs to db https://www.iitism.ac.in/research-cluster-mining-engineering


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmp0a7phj8k.html
Added 9 docs to db https://www.iitism.ac.in/research-cluster-petroleum


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iitism.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Temp file path: /tmp/tmpf7h7u6zn.html
Added 16 docs to db https://www.iitism.ac.in/research-cluster-physics


In [16]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive("chroma_db", 'zip', "chroma_db")

# Download the zip
files.download("chroma_db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
vector_store.similarity_search(
    query="Information Security",
    k=5
)

[Document(metadata={'row': 2, 'source': '/content/1766923003349641_9.csv'}, page_content='Ongoing Projects \n\n \ufeff"Sl. No.": 133\nTitle of the Project: Information Security Education and Awareness (ISEA) Project\nPhase-III\nFunding Agency: Ministry of ElectrBoennigcas la  nd Information\nTechnology, New Delhi\nName of PI: Prof. Sachin Tripathi\nDepartment of PI: Computer Science and Engineering'),
 Document(metadata={'title': 'Information Security', 'url': 'https://www.iitism.ac.in/research-cluster-cse'}, page_content=' Title-Information Security , heading-Not available , url-https://www.iitism.ac.in/research-cluster-cse \n\n description - Prof. Rajendra Pamula'),
 Document(metadata={'title': 'Information Security', 'url': 'https://www.iitism.ac.in/research-cluster-cse'}, page_content=' Title-Information Security , heading-Not available , url-https://www.iitism.ac.in/research-cluster-cse \n\n description - Prof. Hari Om'),
 Document(metadata={'title': 'Information Security', 'url':